## Import Libraries

In [21]:
import os
import time
import random

import timm
import numpy as np
from PIL import Image
from tqdm.notebook import tqdm
# from tqdm import tqdm
from collections import OrderedDict

import torch
import torch.nn as nn
from torch.nn import init
import torch.optim as optim
from torchvision import models
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset

In [22]:
seed = 42

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

## Hyper parameters

In [23]:
# os.environ['CUDA_VISIBLE_DEVICES']='0'
# device = "cpu" 
device = "cuda" if torch.cuda.is_available() else "cpu"
num_epochs = 15
batch_size = 32
lr = 3e-4
gamma = 0.7
unfreeze_after = 2 # unfreeze transformer blocks after 2 epochs
lr_decay = .8
lmbd = 8

## Load Data

In [24]:
transform_train_list = [
    transforms.Resize((224,224), interpolation=3),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]
transform_val_list = [
    transforms.Resize(size=(224,224),interpolation=3), #Image.BICUBIC
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]
data_transforms = {
'train': transforms.Compose( transform_train_list ),
'val': transforms.Compose(transform_val_list),
}

In [25]:
# data_dir = "/home/shubham/CVP/TrainData_split/"
# data_dir = "/home/shubham/CVP/data/"

# image_datasets['train'] = datasets.ImageFolder(os.path.join(data_dir, 'train'),
#                                           data_transforms['train'])
# image_datasets['val'] = datasets.ImageFolder(os.path.join(data_dir, 'val'),
#                                           data_transforms['val'])

In [26]:
train_dir = "/home/shubham/CVP/data/train/"
# val_dir = "/home/shubham/CVP/data/val/all_imgs/"

image_datasets = {}
image_datasets['train'] = datasets.ImageFolder(train_dir, data_transforms['train'])
# image_datasets['val'] = datasets.ImageFolder(val_dir, data_transforms['val'])

train_loader = DataLoader(dataset = image_datasets['train'], batch_size=batch_size, shuffle=True )
# valid_loader = DataLoader(dataset = image_datasets['val'], batch_size=batch_size, shuffle=True)
class_names = image_datasets['train'].classes # '001','003', etc
print(len(class_names))
# print(len(image_datasets['val'].classes)) # '001','003', etc

62


In [27]:
# x,y = next(iter(train_loader))
# print(x.shape, y.shape)

## Model

In [28]:
# weights initialization
def weights_init_kaiming(m):
    classname = m.__class__.__name__
    # print(classname)
    if classname.find('Conv') != -1:
        init.kaiming_normal_(m.weight.data, a=0, mode='fan_in') # For old pytorch, you may use kaiming_normal.
    elif classname.find('Linear') != -1:
        init.kaiming_normal_(m.weight.data, a=0, mode='fan_out')
        init.constant_(m.bias.data, 0.0)
    elif classname.find('BatchNorm1d') != -1:
        init.normal_(m.weight.data, 1.0, 0.02)
        init.constant_(m.bias.data, 0.0)
        
def weights_init_classifier(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        init.normal_(m.weight.data, std=0.001)
        init.constant_(m.bias.data, 0.0)

In [29]:
class FC_Classifier(nn.Module):
    def __init__(self, input_dim, num_classes, droprate=0.5, num_bottleneck=256, return_features=False):
        super(FC_Classifier, self).__init__()
        self.return_features = return_features
        add_block = []
        # if linear:
        add_block += [nn.Linear(input_dim, num_bottleneck)]
        # else:
        #    num_bottleneck = input_dim
        # if bnorm:
        add_block += [nn.BatchNorm1d(num_bottleneck)]
        # if relu:
        #    add_block += [nn.LeakyReLU(0.1)]
        # if droprate > 0:
        add_block+= [nn.Dropout(p=droprate)]
        add_block = nn.Sequential(*add_block)
        add_block.apply(weights_init_kaiming)

        classifier = []
        classifier+= [nn.Linear(num_bottleneck, num_classes)]
        classifier = nn.Sequential(*classifier)
        classifier.apply(weights_init_classifier)

        self.add_block = add_block
        self.classifier = classifier

    def forward(self, x):
        x = self.add_block(x)
        if self.return_features:
            f = x
            x = self.classifier(x)
            return [x,f]
        else:
            x = self.classifier(x)
            return x

In [30]:
class LATransformer(nn.Module):
    def __init__(self, ViT, lmbd, num_classes=751, test=False):
        super(LATransformer, self).__init__()
        self.test = test
        self.class_num = num_classes # output number of classes
        
        # ViT model
        self.model = ViT
        self.model.head.requires_grad_ = False 
        self.cls_token = self.model.cls_token # 1, 1, 768
        self.pos_embed = self.model.pos_embed # 1, 197, 768

        # these are ViT model internal hyper-parameters (FIXED) 
        # self.num_blocks = 12 # number of sequential blocks in ViT
        
        # there are 196 patches in each image; thus, we split them into 14 x 14 grid
        self.num_rows = 14 
        self.num_cols = 14

        # Locally aware network
        self.avgpool = nn.AdaptiveAvgPool2d((self.num_rows,768))
        self.lmbd = lmbd

        if not self.test:
            # ensemble of classifiers
            for i in range(self.num_rows):
                name = 'classifier'+str(i)
                setattr(self, name, FC_Classifier(input_dim=768, num_classes=self.class_num, droprate=0.5, num_bottleneck=256, return_features=False))

    def forward(self, x):
        # x shape = 32, 3, 224, 224
        
        # Divide input image into patch embeddings and add position embeddings
        x = self.model.patch_embed(x) # 32, 196, 768
        cls_token = self.cls_token.expand(x.shape[0], -1, -1)  # 32, 1, 768
        x = torch.cat((cls_token, x), dim=1) # 32, 197, 768
        trnsfrmr_inp = self.model.pos_drop(x + self.pos_embed) # dropout with p = 0; idk!
        
        # Feed forward the x = (patch_embeddings+position_embeddings) through transformer blocks
        # for i in range(self.num_blocks):
        #    x = self.model.blocks[i](x)
        x = self.model.blocks(trnsfrmr_inp)
        x_trnsfrmr_encdd = self.model.norm(x) # layer normalization; shape = 32, 197, 768
        
        # extract the cls token
        cls_token_out = x_trnsfrmr_encdd[:, 0].unsqueeze(1)
        
        # Average pool
        Q = x_trnsfrmr_encdd[:, 1:]
        L = self.avgpool(Q) # 32, 14, 768
        
        if self.test:
            return L
        
        # Add global cls token to each local token 
        for i in range(self.num_rows):
            out = torch.mul(L[:, i, :], self.lmbd)
            L[:,i,:] = torch.div(torch.add(cls_token_out.squeeze(),out), 1+self.lmbd)
        
        # Locally aware network
        part = {}
        predict = {}
        for i in range(self.num_rows):
            part[i] = L[:,i,:] # 32, 768
            name = 'classifier'+str(i)
            c = getattr(self, name)
            predict[i] = c(part[i]) # 32, 751
        return predict

## Load Model

In [31]:
# Load pre-trained ViT
vit_base = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=751)
vit_base = vit_base.to(device)
vit_base.eval()

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU()
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn

In [32]:
# print(torch.cuda.is_available())
# print(type(vit_base))

In [33]:
# Create LA Transformer
model = LATransformer(ViT=vit_base, lmbd=lmbd, num_classes=62).to(device) # len(class_names)
print(model.eval())

LATransformer(
  (model): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU()
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
      (1): Block(
      

### Utilities

In [34]:
def freeze_all_blocks(model):
    # frozen_blocks = 12
    assert len(model.model.blocks) == 12
    for block in model.model.blocks: # [:frozen_blocks]
        for param in block.parameters():
            param.requires_grad=False

def unfreeze_block(model, block_num = 1):
    # unfreeze transformer blocks from last
    for block in model.model.blocks[11-block_num :]:
        for param in block.parameters():
            param.requires_grad=True
    return model

def save_network(network, model_dir, name):
    save_filename = "_best.pth"
    save_path = os.path.join(model_dir, name + save_filename)
    torch.save(network.cpu().state_dict(), save_path)
    
    if torch.cuda.is_available():
        network.cuda()

##  Train

In [35]:
def train_one_epoch(epoch, model, loader, optimizer, loss_fn):
    model.train()
    epoch_accuracy, epoch_loss = 0, 0
    total_samples, correct_predictions = 0, 0
    for data, target in tqdm(loader):
        data, target = data.to(device), target.to(device)

        # predictions
        optimizer.zero_grad()
        output = model(data)
        score = 0.0
        sm = nn.Softmax(dim=1)
        for k, v in output.items():
            score += sm(output[k])
        _, preds = torch.max(score.data, 1)
        
        # backpropagation through ensemble
        loss = 0.0
        for k,v in output.items():
            loss += loss_fn(output[k], target)
        loss.backward()
        optimizer.step()
        
        # print(preds, target.data)
        # acc = (preds == target.data).float().mean()
        # print(acc)
        
        # print(acc)
        # epoch_loss += loss/len(loader)
        # epoch_accuracy += acc / len(loader)
        # if acc:
        #     print(acc, epreds, target.data)
        
        epoch_loss += (loss.item()/data.shape[0])
        correct_predictions += (preds.eq(target.data).sum().item())
        total_samples += data.size(0)
        epoch_accuracy = correct_predictions/total_samples
        # print(f"Epoch : {epoch}; loss : {epoch_loss:.4f}; acc: {epoch_accuracy:.4f}", end="\r")

    # print("total_samples", total_samples, "correct", correct_predictions)
    epoch_loss /= len(loader)
    return OrderedDict([('train_loss', epoch_loss), ("train_accuracy", epoch_accuracy)])

In [36]:
def eval_one_epoch(epoch, model, loader, loss_fn):
    model.eval()
    epoch_accuracy, epoch_loss = 0, 0
    total_samples, correct_predictions = 0, 0
    with torch.no_grad():
        for data, target in tqdm(loader):
            data, target = data.to(device), target.to(device)

            # predictions
            output = model(data)
            score = 0.0
            sm = nn.Softmax(dim=1)
            for k, v in output.items():
                score += sm(output[k])
            _, preds = torch.max(score.data, 1)

            # backpropagation through ensemble
            loss = 0.0
            for k,v in output.items():
                loss += loss_fn(output[k], target)

            epoch_loss += (loss.item()/data.shape[0])
            correct_predictions += (preds.eq(target.data).sum().item())
            total_samples += data.size(0)
            epoch_accuracy = correct_predictions/total_samples
            # print(f"Epoch : {epoch}; loss : {epoch_loss:.4f}; acc: {epoch_accuracy:.4f}", end="\r")

    # print("total_samples", total_samples, "correct", correct_predictions)
    epoch_loss /= len(loader)
    return OrderedDict([('val_loss', epoch_loss), ("val_accuracy", epoch_accuracy)])

In [37]:
model_name = "la-tranformer"
model_dir = "/home/shubham/CVP/model/"
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

In [38]:
freeze_all_blocks(model)
unfreeze_block_id = 0

In [39]:
# loss function
criterion = nn.CrossEntropyLoss()

# optimizer
optimizer = optim.Adam(model.parameters(),weight_decay=5e-4, lr=lr)

# # scheduler
# scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

In [40]:
print("training...")
# num_eps = 10
# pbar = tqdm(np.arange(num_eps).tolist())
for epoch in range(num_epochs):

    if epoch % unfreeze_after == 0: # and epoch != 0:
        unfreeze_block_id += 1
        model = unfreeze_block(model, unfreeze_block_id)
        optimizer.param_groups[0]['lr'] *= lr_decay 
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        # print(f"Unfrozen Blocks: {unfreeze_block_id}, Current lr: {optimizer.param_groups[0]['lr']}, Trainable Params: {trainable_params}")

    train_metrics = train_one_epoch(epoch, model, train_loader, optimizer, criterion)
#     val_metrics = eval_one_epoch(epoch, model, valid_loader, criterion)
    ta = train_metrics['train_accuracy']
    tl = train_metrics['train_loss']
#     va = val_metrics['val_accuracy']
#     vl = val_metrics['val_loss']
#     pbar.set_description(f"Train Acc : {ta}, Train Loss : {tl}, Val Acc : {va}, Val Loss : {vl}")
    
    print(f"Epoch : {epoch}; trainacc : {ta:.4f}")
#     print(f"Epoch : {epoch}; trainacc : {ta:.4f}; valacc: {va:.4f}", end="\r")

#     deep copy the model
#     last_model_wts = model.state_dict()
#     if eval_metrics['val_accuracy'] > best_acc:
#         best_acc = eval_metrics['val_accuracy']
#         save_network(model, epoch,name)
#         print("SAVED!")

training...


  0%|          | 0/31 [00:00<?, ?it/s]

Epoch : 0; trainacc : 0.2399


  0%|          | 0/31 [00:00<?, ?it/s]

Epoch : 1; trainacc : 0.5131


...

Epoch : 12; trainacc : 0.9879


  0%|          | 0/31 [00:00<?, ?it/s]

Epoch : 13; trainacc : 0.9960


  0%|          | 0/31 [00:00<?, ?it/s]

Epoch : 14; trainacc : 0.9980


In [41]:
save_network(model, model_dir, model_name)

### Appendix

In [42]:
# vit_base

In [43]:
# x,y = next(iter(train_loader))
# print(x.shape, y.shape)

In [44]:
# print(x.shape)
# x = vit_base.patch_embed(x)
# print(x.shape)
# print()

# print(vit_base.cls_token.shape, vit_base.pos_embed.shape)
# cls_token = vit_base.cls_token.expand(x.shape[0], -1, -1) 
# print(cls_token.shape)
# x = torch.cat((cls_token, x), dim=1)
# print(x.shape)
# x = vit_base.pos_drop(x + vit_base.pos_embed)
# print(x.shape)
# print()

# # Feed forward the x = (patch_embeddings+position_embeddings) through transformer blocks
# # for i in range(12):
# x = vit_base.blocks(x)
# x = vit_base.norm(x) # layer normalization
# print(x.shape)

In [45]:
# # extract the cls token
# cls_token_out = x[:, 0].unsqueeze(1)
# print(cls_token_out.shape)

# # Average pool
# avgpool = nn.AdaptiveAvgPool2d(output_size = (14, 768))
# print(x.shape)
# x = avgpool(x[:, 1:]) # input is 32,196,768
# print(x.shape)